In [16]:
import openai
import os


# Set API key
openai.api_key = ""

In [19]:

## Set the API key and model name
MODEL="gpt-4o-mini"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

In [21]:
# Step 1: 使用gpt-4o-mini
step1_input = """
目标：

我需要你去帮助我构思并且梳理观点，它的主题是：

-

核心需要解决的问题：如何写专利，限制在100个token内。

首先，请毕业于Harvard的麦肯锡咨询专家，揣摩我的观点，帮助我思考，提出有效的问题并完善这个点子。带有强的逻辑性，创造性，以及批判性思维。
"""

response_step1 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": step1_input}
    ],
    max_tokens=200
)

step1_output = response_step1.choices[0].message.content.strip()
print("Step 1 Output:", step1_output)

Step 1 Output: 撰写专利的关键在于清晰定义创新点、确保新颖性和实用性。首先，明确你的发明是什么，解决了什么问题。其次，进行全面的专利检索，确认无相似专利。在撰写时，使用规范语言，详尽描述技术细节和应用场景，确保可理解性。此外，合理组织结构，包含背景、发明简介、权利要求等部分。

问题：
1


In [25]:
import openai
import os

# Set the API key and model name
MODEL = "gpt-4o-mini"

# Define debug mode
DEBUG_MODE = False

def get_completion(model, messages, max_tokens=100):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens
        )
        content = response.choices[0].message.content.strip()
        print(f"Response: {content}")
        return content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return None

def main(user_input):
    max_tokens = 100 if DEBUG_MODE else 1500
    memory = ""

    # Step 1: 使用gpt-4o-mini
    step1_input = f"""
    目标：

    我需要你去帮助我构思并且梳理观点，它的主题是：

    {user_input}

    核心需要解决的问题：如何写专利，限制在100个token内。

    首先，请毕业于Harvard的麦肯锡咨询专家，揣摩我的观点，帮助我思考，提出有效的问题并完善这个点子。带有强的逻辑性，创造性，以及批判性思维。
    """
    step1_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking."},
        {"role": "user", "content": step1_input}
    ]
    step1_output = get_completion(MODEL, step1_messages, max_tokens=max_tokens)
    print(f"Step 1 Output: {step1_output}")

    memory += f"Step 1 Output: {step1_output}\n"

    # Step 2: 使用gpt-4o
    step2_input = f"""
    首先，请顶尖的OpenAI prompt engineer帮助我思考，提出有效的问题，并且挖掘新的专利点。

    不着急，think step by step, 加入你的思考逻辑，思考出三个创新点。请专家团队选择最佳的创新点，然后写出详细内容。

    Step 1 Output:
    {step1_output}
    """
    step2_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking. Use the following memory to provide context for the user's request."},
        {"role": "user", "content": step2_input}
    ]
    step2_output = get_completion("gpt-4o", step2_messages, max_tokens=max_tokens)
    print(f"Step 2 Output: {step2_output}")

    memory += f"Step 2 Output: {step2_output}\n"

    # Step 3: 使用gpt-4o-mini
    step3_input = f"""
    从以下三个创新点中挑选一个，然后写出详细内容：

    {step2_output}

    请详细描述你选择的创新点。
    """
    step3_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking. Use the following memory to provide context for the user's request."},
        {"role": "user", "content": step3_input}
    ]
    step3_output = get_completion(MODEL, step3_messages, max_tokens=max_tokens)
    print(f"Step 3 Output: {step3_output}")

    memory += f"Step 3 Output: {step3_output}\n"

    # Step 4: 使用gpt-4o
    step4_input = f"""
    针对以下问题解决方案：为Mercedes Benz写出customized patent，优化，加入Mercedes data scientist和research scientist的理解，Tech Lead加入更多技术实现和算法的细节。将这部分进一步的细节公式化，模型化，以便patent不会被查出重复。
    加入数学公式至关重要
    问题解决方案：

    {step3_output}
    """
    step4_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking. Use the following memory to provide context for the user's request."},
        {"role": "user", "content": step4_input}
    ]
    step4_output = get_completion("gpt-4o", step4_messages, max_tokens=max_tokens)
    print(f"Step 4 Output: {step4_output}")

    # 保存到文件
    with open('basic_information.txt', 'w', encoding='utf-8') as f:
        f.write(step3_output if step3_output else '')

    with open('technical_details.txt', 'w', encoding='utf-8') as f:
        f.write(step4_output if step4_output else '')

if __name__ == "__main__":
    user_input = """**难点：**

1. 信号无法传输的场景下，车上传输数据到云端的更新会有延迟，导致常规逻辑的增量更新无法解决补发数据的问题

**解决方案：**

1. 查询data maturity rate, 通过是否为补发数据的列，来查询补发数据进入云的时间，制作时间matrix
2. 时间matrix output: 多少天 可以达到99.999%补发数据存入云，获取数据稳定天数 
3. 通过delta （parquet with logs) 的特性，以及功能time travel 以及  upsert 识别数据是否为补发来进行数据替换
4. 使用delta 文件的 vaccum 来清楚历史version的数据

首先，请毕业于stanford 的Databricks world class data engineer，揣摩我的观点，帮助我思考，提出有效的问题并完善这个点子。带有强的逻辑性，创造性，以及批判性思维。"""
    main(user_input)


Response: ### 针对您的主题和解决方案的思考与建议：

#### 1. 保证数据的实时性
- **问题**：在信号无法传输的情况下，如何确保补发数据的及时整合与更新？
  - **建议**：考虑采用分层存储（hot and cold storage），以便按优先级处理补发数据。

#### 2. 数据的准确性与完整性
- **问题**：如何确保补发数据不与原数据产生冲突？
  - **建议**：实现数据完整性校验机制，在补发数据进入云端之前，验证其有效性与一致性。

#### 3. 时间矩阵的可行性
- **问题**：时间矩阵是否能够灵活应对不同场景下的延迟？
  - **建议**：设计动态的时间阈值，根据网络状况实时调整更新频率。

#### 4. Delta Lake的应用
- **问题**：在使用Delta Lake进行数据替换时，是否考虑数据一致性？
  - **建议**：引入版本控制机制，确保上层数据在替换时不会被损坏。

#### 5. 数据清理与管理
- **问题**：如何有效执行数据清理以提高存储效率？
  - **建议**：设定定期的自动清理策略，结合数据使用频率进行优化。

### 方案总结：

在专利撰写时，可以集中描述上述措施的创新性和实用性。建议一个清晰的结构，以背景、问题、解决方案、实施细节和预期结果为框架，确保符合科技专利的标准要求。

### 专利写作核心要点（100个token）：

本发明涉及一种在信号中断时，提升车辆数据云端更新的系统和方法，包括一个动态时间矩阵，用于评估补发数据存入云端所需时间；采用Delta Lake技术，通过检验数据状态与清理历史版本以保证数据准确性与一致性。此方法有效解决数据更新延迟，实现实时可靠的数据管理。
Step 1 Output: ### 针对您的主题和解决方案的思考与建议：

#### 1. 保证数据的实时性
- **问题**：在信号无法传输的情况下，如何确保补发数据的及时整合与更新？
  - **建议**：考虑采用分层存储（hot and cold storage），以便按优先级处理补发数据。

#### 2. 数据的准确性与完整性
- **问题**：如何确保补发数据不与原数据产生冲突？
  - **建议**：实现数据完整性校验机制，在补发数据进入云端之前，验证其有效性与一致性。


In [ ]:
import openai
import os

# Set the API key and model name
MODEL = "gpt-4o-mini"
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY",""))

# Define debug mode
DEBUG_MODE = True

def get_completion(model, messages, max_tokens=100):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens
        )
        content = response.choices[0].message.content.strip()
        print(f"Response: {content}")
        return content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return None

def main(user_input):
    max_tokens = 100 if DEBUG_MODE else 1500
    memory = ""

    # Step 1: Use gpt-4o-mini
    step1_input = f"""
    Goal:

    I need you to help me brainstorm and organize ideas. The topic is:

    {user_input}

    The core issue to be addressed: How to write a patent, limited to 100 tokens.

    First, please channel a Harvard graduate and McKinsey consultant to refine my idea, ask effective questions, and improve this concept. It should have strong logic, creativity, and critical thinking.
    """
    step1_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking."},
        {"role": "user", "content": step1_input}
    ]
    step1_output = get_completion(MODEL, step1_messages, max_tokens=max_tokens)
    print(f"Step 1 Output: {step1_output}")

    memory += f"Step 1 Output: {step1_output}\n"

    # Step 2: Use gpt-4o
    step2_input = f"""
    Next, please channel a top OpenAI prompt engineer to help me think, ask effective questions, and explore new patent ideas.

    Take your time, think step by step, include your thought process, and describe three innovative ideas.

    Step 1 Output:
    {step1_output}
    """
    step2_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking. Use the following memory to provide context for the user's request."},
        {"role": "user", "content": step2_input}
    ]
    step2_output = get_completion("gpt-4o", step2_messages, max_tokens=max_tokens)
    print(f"Step 2 Output: {step2_output}")

    memory += f"Step 2 Output: {step2_output}\n"

    # Step 3: Use gpt-4o-mini
    step3_input = f"""
    Choose one of the following three innovative ideas and write a detailed description:

    {step2_output}

    Please describe the chosen innovative idea in detail.
    """
    step3_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking. Use the following memory to provide context for the user's request."},
        {"role": "user", "content": step3_input}
    ]
    step3_output = get_completion(MODEL, step3_messages, max_tokens=max_tokens)
    print(f"Step 3 Output: {step3_output}")

    memory += f"Step 3 Output: {step3_output}\n"

    # Step 4: Use gpt-4o
    step4_input = f"""
    Based on the following solution: Write a customized patent for Mercedes Benz, optimize it, include insights from Mercedes data scientists and research scientists, and have a Tech Lead add more technical implementation details and algorithms. Further formalize and model these details so that the patent cannot be found to be redundant.

    Problem solution:

    {step3_output}
    """
    step4_messages = [
        {"role": "system", "content": "You are a helpful assistant with expertise in patent writing and critical thinking. Use the following memory to provide context for the user's request."},
        {"role": "user", "content": step4_input}
    ]
    step4_output = get_completion("gpt-4o", step4_messages, max_tokens=max_tokens)
    print(f"Step 4 Output: {step4_output}")

    # Save to files
    with open('basic_information.txt', 'w', encoding='utf-8') as f:
        f.write(step3_output if step3_output else '')

    with open('technical_details.txt', 'w', encoding='utf-8') as f:
        f.write(step4_output if step4_output else '')

if __name__ == "__main__":
    user_input = "This is the user input for the patent topic."
    main(user_input)
